# Extract main color from images

Try using algolia color extractor to identify the color of the primary image in the poster.

## Color-Extractor -- DID NOT WORK

In [2]:
!pip3 install -r /home/ubuntu/requirements.txt

  Using cached cycler-0.10.0-py2.py3-none-any.whl
  Using cached dask-0.10.1-py2.py3-none-any.whl
  Using cached decorator-4.0.10-py2.py3-none-any.whl
  Using cached ijson-2.3-py2.py3-none-any.whl
  Using cached matplotlib-1.5.1.tar.gz
  Using cached networkx-1.11-py2.py3-none-any.whl
    100% |████████████████████████████████| 15.6MB 103kB/s eta 0:00:01
    100% |████████████████████████████████| 5.6MB 294kB/s eta 0:00:01
  Using cached pyparsing-2.1.5-py2.py3-none-any.whl
  Using cached python_dateutil-2.5.3-py2.py3-none-any.whl
  Using cached pytz-2016.6.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 27.9MB 56kB/s  eta 0:00:01
    100% |████████████████████████████████| 17.6MB 90kB/s  eta 0:00:01  9% |███                             | 1.6MB 24.6MB/s eta 0:00:01    23% |███████▌                        | 4.1MB 18.8MB/s eta 0:00:01
    100% |████████████████████████████████| 41.1MB 35kB/s  eta 0:00:01
  Using cached six-1.10.0-py2.py3-none-any.whl
  Using cached too

In [11]:
!pip3 install opencv-python

    100% |████████████████████████████████| 6.7MB 240kB/s eta 0:00:01
  Using cached numpy-1.12.1-cp35-cp35m-manylinux1_x86_64.whl
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import cv2
import numpy as np

from color_extractor import ImageToColor



/home/ubuntu/.local/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/ubuntu/.local/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
npz = np.load('/home/ubuntu/color_names.npz')
img_to_color = ImageToColor(npz['samples'], npz['labels'])

img = cv2.imread('/home/ubuntu/Posters/104677_1.jpg')
print(img_to_color.get(img))

KeyError: 'debug'

## Color-Thief -- WORKED!

In [3]:
!pip3 install colorthief

    100% |████████████████████████████████| 5.7MB 280kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 13.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for olefile ... - done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/20/58/49/cc7bd00345397059149a10b0259ef38b867935ea2ecff99a9b
Successfully built olefile
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from colorthief import ColorThief

In [2]:
color_thief = ColorThief('/home/ubuntu/Posters/104677_1.jpg')
# get the dominant color
dominant_color = color_thief.get_color(quality=1)
# build a color palette
palette = color_thief.get_palette(color_count=6)

In [3]:
dominant_color

(135, 143, 150)

In [7]:
!pip3 install webcolors

  Using cached webcolors-1.7.tar.gz
  Running setup.py bdist_wheel for webcolors ... - done
  Stored in directory: /home/ubuntu/.cache/pip/wheels/4a/bb/ba/464319914f0bfb826beb0955bbb29d7188a18368ea226a758d
Successfully built webcolors
You are using pip version 8.1.1, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
import webcolors as wc

In [11]:
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.css3_hex_to_names.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]

def get_colour_name(requested_colour):
    try:
        closest_name = actual_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
        actual_name = None
    return actual_name, closest_name

In [12]:
get_colour_name(dominant_color)

(None, 'lightslategray')

In [17]:
get_colour_name(dominant_color)[1]

'lightslategray'

In [15]:
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img, list_pictures

Using TensorFlow backend.


In [16]:
path = '/home/ubuntu/Posters/'
posters = list_pictures(path)
posters[:3]

['/home/ubuntu/Posters/312809_1.jpg',
 '/home/ubuntu/Posters/36938_1.jpg',
 '/home/ubuntu/Posters/400140_1.jpg']

In [19]:
ids = []
rgbs = []
colors = []

for poster in posters:
    p_id = str(poster[21:])
    img = ColorThief(poster)
    rgb_color = img.get_color(quality=1)
    color = get_colour_name(rgb_color)[1]
    ids.append(p_id)
    rgbs.append(rgb_color)
    colors.append(color)

In [24]:
import pandas as pd

In [26]:
comb = pd.DataFrame({'filename': ids, 'primary_color': colors})
comb.head()

,filename,primary_color
0,312809_1.jpg,salmon
1,36938_1.jpg,black
2,400140_1.jpg,darkslategray
3,114236_1.jpg,lightgray
4,37944_1.jpg,silver


In [30]:
data = pd.read_csv('/home/ubuntu/150samples_metadata.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)
data.head(2)

,genre_ids,ids,popularity,release_date,release_year,title,vote_average,url poster1,filename
0,28,400140,1.000898,2010-09-30,2010,Caos,0.0,http://image.tmdb.org/t/p/w500/nqk1cKEKvkpHHeM...,400140_1.jpg
1,28,187682,1.034173,1975-06-07,1975,Lady of the Law,6.0,http://image.tmdb.org/t/p/w500/eA5U2DNNfw674Gv...,187682_1.jpg


In [31]:
all_info = pd.merge(data, comb, how='inner', on='filename')
all_info.head()

,genre_ids,ids,popularity,release_date,release_year,title,vote_average,url poster1,filename,primary_color
0,28,400140,1.000898,2010-09-30,2010,Caos,0.0,http://image.tmdb.org/t/p/w500/nqk1cKEKvkpHHeM...,400140_1.jpg,darkslategray
1,28,187682,1.034173,1975-06-07,1975,Lady of the Law,6.0,http://image.tmdb.org/t/p/w500/eA5U2DNNfw674Gv...,187682_1.jpg,palevioletred
2,28,27172,1.332542,1970-09-18,1970,There Was a Crooked Man...,6.9,http://image.tmdb.org/t/p/w500/nnaKoENm9RslQIt...,27172_1.jpg,linen
3,28,202182,1.006750,1974-05-21,1974,Young Lovers on Flying Wheels,0.0,http://image.tmdb.org/t/p/w500/c25g5bNXSZDHdHu...,202182_1.jpg,burlywood
4,28,251531,1.000384,1978-02-09,1978,The Hero Tattooed with Nine Dragons,0.0,http://image.tmdb.org/t/p/w500/6DK1x2tuiCHeFkE...,251531_1.jpg,chocolate


In [32]:
all_info.to_csv('/home/ubuntu/150samples_metadata_color.csv')